In [ ]:
#get necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:
#get keras
import keras
from keras.applications import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.layers import *
from keras import Model, Sequential
from keras.models import load_model

In [ ]:
#Load data
y = np.load('Y_train.npy')

In [ ]:
#do not standardize images
X = np.load('X_train.npy')/255.

In [ ]:
X = X.reshape((X.shape[0],X.shape[1],X.shape[2],1))

In [ ]:
X.shape, y.shape

In [ ]:
#set seed
np.random.seed(111)

#split to test, train and val
s = np.arange(X.shape[0])
np.random.shuffle(s)
s_train = s[:8000]
s_val = s[8000:9000]
s_test =s[9000:X.shape[0]]

In [ ]:
#split into train and val
X_train = X[s_train]
y_train = y[s_train]

X_val = X[s_val]
y_val = y[s_val]

#generate testset
X_test = X[s_test]
y_test = y[s_test]

In [ ]:
#convert y in y_bin
y_bin = np.zeros((y.shape[0],1))

In [ ]:
#convert y to y_bin
#0 - normal 1 - pathological
for i in range(y.shape[0]):
    if all(y[i]==0):
        y_bin[i] = 0
    else:
        y_bin[i] = 1
            
        

In [ ]:
#split y into train and test
y_bin_train = y_bin[s_train]
y_bin_val = y_bin[s_val]
y_bin_test = y_bin[s_test]

In [ ]:
#check y_bin traintest and val sets are balanced
print("ratio of normal to total in train:",len(np.where(y_bin_train==0)[0])/y_bin_train.shape[0])
print("ratio of normal to total in val:",len(np.where(y_bin_val==0)[0])/y_bin_val.shape[0])
print("ratio of normal to total in test:",len(np.where(y_bin_test==0)[0])/y_bin_test.shape[0])

- They seem adequately balanced

In [ ]:
#Testing out alorithm with black and whites
## creating dummy black and white images
# X_train_ = np.zeros_like(X_train)
# for i,y in enumerate(y_bin_train):
#     X_train_[i] = np.ones((256, 256, 1))*y

In [ ]:
shape, classes = (256, 256, 1), 1
input_tnsr = keras.layers.Input(shape)

In [ ]:
#Set learning rate for all models 0.0001 (showed to be the best rate)
adam_customlr = keras.optimizers.Adam(lr=0.0001) #decrease lr

In [ ]:
#Set Epoch number for all
num_epoch=1

## Method
In the following we will compare three models: A simple model, resnet50, MobileNet and VGG16 as the literature suggests these perform best. We will do so by keeping the parameters constant to select the best performing architecture and then choose the winning model in terms of accuracy to perform fine tuning.

## Simple Model

In [ ]:
#create simple model; 6 layers
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = adam_customlr, loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#fit the model
history_simple_model = model.fit(X_train, y_bin_train, epochs=num_epoch, batch_size=32, verbose=1,validation_data=(X_val,y_bin_val))

In [ ]:
#Baseline model - Visualize accuracy and loss of test and validation
print(history_simple_model.history.keys())
# summarize history for accuracy
plt.plot(history_simple_model.history['acc'])
plt.plot(history_simple_model.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_simple_model.history['loss'])
plt.plot(history_simple_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Accuracy and loss on test set for baseline model
model.evaluate(X_test,y_bin_test)

In [ ]:
#save simple model as the baseline
model.save('simple_model.h5')

## Resnet-Model (50 layers)

In [ ]:
# Simple resnet - no data augmentation or pretraining. 
input_tnsr = keras.layers.Input(shape)
model_rsnt = ResNet50(include_top=False, weights = None, input_tensor = input_tnsr, classes = classes, pooling=None)
rsnt_input_layer = model_rsnt.input

In [ ]:
x = model_rsnt.layers[-1].output
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

cstm_resnet = Model(inputs=rsnt_input_layer, outputs=x)

In [ ]:
cstm_resnet.summary()

In [ ]:
#compile and fit- same learning rate as simple model
cstm_resnet.compile(optimizer = adam_customlr, loss="binary_crossentropy", metrics=["accuracy"])
history_cstm_resnet = cstm_resnet.fit(X_train, y_bin_train, epochs=num_epoch, batch_size=32, verbose=1,validation_data=(X_val,y_bin_val))

In [ ]:
#accuracy and loss on test set 
cstm_resnet.evaluate(X_test,y_bin_test)

In [ ]:
#generate plots
#Baseline model
# list all data in history
print(history_cstm_resnet.history.keys())
# summarize history for accuracy
plt.plot(history_cstm_resnet.history['acc'])
plt.plot(history_cstm_resnet.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_cstm_resnet.history['loss'])
plt.plot(history_cstm_resnet.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#save simple model as the cstm_resnet
cstm_resnet.save('cstm_resnet.h5')

## MobileNet

In [ ]:
#Loading MobileNet
from keras.applications.mobilenet import MobileNet
mobnet = MobileNet(input_tensor=input_tnsr, include_top=False, weights=None, classes=classes, pooling=None)

In [ ]:
mobnet_input_layer = mobnet.input

In [ ]:
x1 = mobnet.layers[-1].output
x1 = Flatten()(x1)
x1 = Dense(32, activation='relu')(x1)
x1 = Dense(1, activation='sigmoid')(x1)

cstm_mobnet = Model(inputs=mobnet_input_layer, outputs=x1)

In [ ]:
cstm_mobnet.summary()

In [ ]:
#compile and fit- same learning rate as simple model;
cstm_mobnet.compile(optimizer = adam_customlr, loss="binary_crossentropy", metrics=["accuracy"])
history_cstm_mobnet = cstm_mobnet.fit(X_train, y_bin_train, epochs=num_epoch, batch_size=32, verbose=1,validation_data=(X_val,y_bin_val))

In [ ]:
#accuracy and loss on test set 
cstm_mobnet.evaluate(X_test,y_bin_test)[1]

In [ ]:
#generate plots
#Baseline model
# list all data in history
print(history_cstm_mobnet.history.keys())
# summarize history for accuracy
plt.plot(history_cstm_mobnet.history['acc'])
plt.plot(history_cstm_mobnet.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_cstm_mobnet.history['loss'])
plt.plot(history_cstm_mobnet.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#save simple model as the cstm_resnet
cstm_mobnet.save('cstm_mobnet.h5')

## VGG16

In [ ]:
#try out VGG16
from keras.applications.vgg16 import VGG16

In [ ]:
input_tnsr = keras.layers.Input(shape)

In [ ]:
cstm_vgg = VGG16(include_top=False, weights = None,  input_tensor = input_tnsr, classes = classes, pooling=None)
vgg_input_layer = model_vgg.input

In [ ]:
x2 = model_vgg.layers[-1].output
x2 = Flatten()(x2)
x2 = Dense(32, activation='relu')(x2)
x2 = Dense(1, activation='sigmoid')(x2)

cstm_vgg = Model(inputs=vgg_input_layer, outputs=x2)

In [ ]:
cstm_vgg.summary()

In [ ]:
#compile
cstm_vgg.compile(optimizer = adam_customlr, loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#compile and fit- same learning rate as simple model; only change epochs to 15
history_cstm_vgg = cstm_vgg.fit(X_train, y_bin_train, epochs=num_epoch, batch_size=32, verbose=1,validation_data=(X_val,y_bin_val))

In [ ]:
#generate plots
#Baseline model
# list all data in history
print(history_cstm_vgg.history.keys())
# summarize history for accuracy
plt.plot(history_cstm_vgg.history['acc'])
plt.plot(history_cstm_vgg.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_cstm_vgg.history['loss'])
plt.plot(history_cstm_vgg.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#accuracy and loss on test set 
cstm_vgg.evaluate(X_test,y_bin_test)

In [ ]:
#save simple model as the cstm_resnet
cstm_vgg.save('cstm_vgg.h5')

## Results of comparison
The best performing architecture seems to be the VGG16. We will proceed with it and try different kind of architectures. 

## VGG with imagenet weights

In [ ]:
X_raw = np.load('X_train.npy')/255.

In [ ]:
X_raw.shape

In [ ]:
X_3ch= np.zeros((X_raw.shape[0],X_raw.shape[1],X_raw.shape[2],3))

In [ ]:
#copying grayscale images across 3 channel for  using imagenet-trained weights 
for i in range(X_raw.shape[0]):
    X_3ch[i] = np.stack((X_raw[i],)*3, -1)


In [ ]:
X_3ch.shape

In [ ]:
X_3c_train = X_3ch[s_train]
X_3c_val = X_3ch[s_val]
X_3c_test = X_3ch[s_test]

In [ ]:
#new shape and tensor
shape_3c = (256, 256, 3)
input_tnsr_3c = keras.layers.Input(shape_3c)

In [ ]:
#change weights to imagenet
model_vgg_weights = VGG16(include_top=False, weights = 'imagenet', input_tensor = input_tnsr_3c, classes = classes, pooling=None)
vgg_im_input_layer = model_vgg_weights.input



In [ ]:
x3 = model_vgg_weights.layers[-1].output
x3 = Flatten()(x3)
x3 = Dense(32, activation='relu')(x3)
x3 = Dense(1, activation='sigmoid')(x3)

model_vgg_weights = Model(inputs=vgg_im_input_layer, outputs=x3)

In [ ]:
model_vgg_weights.summary()

In [ ]:
#compile and fit- same learning rate as simple model; only change epochs to 15
model_vgg_weights.compile(optimizer = adam_customlr, loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history_cstm_vgg_weights = model_vgg_weights.fit(X_3c_train, y_bin_train, epochs=num_epoch, batch_size=32, verbose=1,validation_data=(X_3c_val,y_bin_val))

In [ ]:
#accuracy and loss on test set 
model_vgg_weights.evaluate(X_3c_test,y_bin_test)

In [ ]:
#generate plots
#Baseline model
# list all data in history
print(history_cstm_vgg_weights.history.keys())
# summarize history for accuracy
plt.plot(history_cstm_vgg_weights.history['acc'])
plt.plot(history_cstm_vgg_weights.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_cstm_vgg_weights.history['loss'])
plt.plot(history_cstm_vgg_weights.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#save simple model as the cstm_resnet
model_vgg_weights.save('cstm_vgg_weights.h5')

#### VGG with data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
  #  featurewise_center=True,
  #  featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen=ImageDataGenerator()

In [ ]:
vgg16_aug = cstm_vgg

In [ ]:
vgg16_aug.compile(optimizer = adam_customlr, loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# fits the model on batches with real-time data augmentation:
vgg16_aug_history = cstm_vgg.fit_generator(datagen.flow(X_train, y_bin_train, batch_size=32),
                        steps_per_epoch=len(X_train) // 32, epochs=num_epoch, verbose=1,
                                   validation_data=test_datagen.flow(X_val,y_bin_val,batch_size=32))


In [ ]:
#generate plots
#Baseline model
# list all data in history
print(vgg16_aug_history.history.keys())
# summarize history for accuracy
plt.plot(vgg16_aug_history.history['acc'])
plt.plot(vgg16_aug_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(vgg16_aug_history.history['loss'])
plt.plot(vgg16_aug_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
vgg16_aug.evaluate(X_test,y_bin_test)

In [ ]:
#save simple model as the cstm_resnet
vgg16_aug.save('vgg16_aug.h5')

## Computing ROC for different models

### ROC for simple model

In [ ]:
#plotting ROC curves
model = load_model('baseline.h5')

In [ ]:
model.evaluate(X_test,y_bin_test)

In [ ]:
y_pred = model.predict(X_test).ravel()

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import roc_curve

fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_bin_test, y_pred)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()